# Lab 14: Your First AI Agent

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/depalmar/ai_for_the_win/blob/main/notebooks/lab14_first_ai_agent.ipynb)

Build a simple ReAct agent from scratch to understand how AI agents work.

## Learning Objectives
- Understand the ReAct (Reason + Act) pattern
- Build a simple agent loop from scratch
- Implement tool calling for security tasks
- Debug agent behavior and handle edge cases

## What is an AI Agent?

An AI agent is an LLM that can:
1. **Reason** about a task
2. **Act** by calling tools
3. **Observe** the results
4. **Repeat** until the task is done

**Next:** Lab 36 (Threat Intel Agent)

In [ ]:
#@title Install dependencies (Colab only)
#@markdown Run this cell to install required packages in Colab

%pip install -q anthropic openai google-generativeai python-dotenv

In [ ]:
#@title LLM Setup (Provider-Agnostic)
#@markdown Set your API key in Colab Secrets (🔑 icon in sidebar)

import os
import json
import re
from typing import Dict, List, Any, Callable

# Try to load from Colab secrets
try:
    from google.colab import userdata
    for key in ["ANTHROPIC_API_KEY", "OPENAI_API_KEY", "GOOGLE_API_KEY"]:
        try:
            os.environ[key] = userdata.get(key)
        except:
            pass
except:
    pass  # Not in Colab, use environment variables

def setup_llm():
    """Detect and configure LLM provider."""
    if os.environ.get("ANTHROPIC_API_KEY"):
        return "anthropic", "claude-sonnet-4.5"
    elif os.environ.get("OPENAI_API_KEY"):
        return "openai", "gpt-5"
    elif os.environ.get("GOOGLE_API_KEY"):
        return "google", "gemini-3-flash"
    raise ValueError("No API key found. Add one to Colab Secrets.")

def query_llm(prompt: str, system: str = "You are a security analyst.") -> str:
    """Query the configured LLM."""
    provider, model = setup_llm()
    
    if provider == "anthropic":
        from anthropic import Anthropic
        client = Anthropic()
        response = client.messages.create(
            model=model, max_tokens=2048, system=system,
            messages=[{"role": "user", "content": prompt}]
        )
        return response.content[0].text
    elif provider == "openai":
        from openai import OpenAI
        client = OpenAI()
        response = client.chat.completions.create(
            model=model, max_tokens=2048,
            messages=[{"role": "system", "content": system}, {"role": "user", "content": prompt}]
        )
        return response.choices[0].message.content
    elif provider == "google":
        import google.generativeai as genai
        genai.configure(api_key=os.environ.get("GOOGLE_API_KEY"))
        model_instance = genai.GenerativeModel(model)
        response = model_instance.generate_content(f"{system}\n\n{prompt}")
        return response.text

provider, model = setup_llm()
print(f"✅ Using {provider} ({model})")

## Step 1: Define Our Tools

Tools are functions the agent can call. Let's create some security-focused tools.

In [ ]:
# Mock threat intelligence database
THREAT_INTEL_DB = {
    "192.168.1.100": {"malicious": False, "category": "internal"},
    "45.33.32.156": {"malicious": True, "category": "c2_server", "actor": "APT28"},
    "evil.com": {"malicious": True, "category": "phishing"},
    "google.com": {"malicious": False, "category": "legitimate"},
}

def lookup_ip(ip: str) -> Dict:
    """Look up an IP address in threat intelligence."""
    if ip in THREAT_INTEL_DB:
        return THREAT_INTEL_DB[ip]
    return {"malicious": False, "category": "unknown", "note": "Not in database"}

def lookup_domain(domain: str) -> Dict:
    """Look up a domain in threat intelligence."""
    if domain in THREAT_INTEL_DB:
        return THREAT_INTEL_DB[domain]
    return {"malicious": False, "category": "unknown", "note": "Not in database"}

def check_hash(file_hash: str) -> Dict:
    """Check if a file hash is known malware."""
    # Mock implementation
    known_malware = {
        "abc123": {"malicious": True, "family": "Emotet"},
        "def456": {"malicious": True, "family": "Cobalt Strike"},
    }
    return known_malware.get(file_hash, {"malicious": False, "note": "Unknown hash"})

# Tool registry
TOOLS = {
    "lookup_ip": lookup_ip,
    "lookup_domain": lookup_domain,
    "check_hash": check_hash,
}

print("✅ Tools defined:", list(TOOLS.keys()))

## Step 2: Build the Agent Loop

The agent loop is the core of any AI agent. It:
1. Asks the LLM what to do
2. Parses the response for tool calls
3. Executes the tool
4. Feeds the result back to the LLM
5. Repeats until done

In [ ]:
AGENT_SYSTEM_PROMPT = """
You are a security analyst agent. You can use tools to investigate security questions.

Available tools:
- lookup_ip(ip): Look up an IP address in threat intelligence
- lookup_domain(domain): Look up a domain in threat intelligence  
- check_hash(hash): Check if a file hash is known malware

To use a tool, respond with:
THOUGHT: [your reasoning]
ACTION: tool_name("argument")

After receiving tool results, continue reasoning.

When you have enough information, respond with:
THOUGHT: [final reasoning]
FINAL ANSWER: [your conclusion]
"""

def parse_agent_response(response: str) -> tuple:
    """Parse agent response to extract action or final answer."""
    # Check for final answer
    if "FINAL ANSWER:" in response:
        answer = response.split("FINAL ANSWER:")[-1].strip()
        return ("final", answer)
    
    # Check for action
    action_match = re.search(r'ACTION:\s*(\w+)\("([^"]+)"\)', response)
    if action_match:
        tool_name = action_match.group(1)
        argument = action_match.group(2)
        return ("action", tool_name, argument)
    
    return ("error", "Could not parse response")

def run_agent(query: str, max_steps: int = 5) -> str:
    """Run the agent loop."""
    conversation = f"User Query: {query}\n\n"
    
    for step in range(max_steps):
        print(f"\n--- Step {step + 1} ---")
        
        # Get LLM response
        response = query_llm(conversation, AGENT_SYSTEM_PROMPT)
        print(f"Agent: {response[:500]}..." if len(response) > 500 else f"Agent: {response}")
        
        # Parse response
        parsed = parse_agent_response(response)
        
        if parsed[0] == "final":
            print(f"\n✅ Final Answer: {parsed[1]}")
            return parsed[1]
        
        elif parsed[0] == "action":
            tool_name, argument = parsed[1], parsed[2]
            print(f"\n🔧 Calling tool: {tool_name}({argument})")
            
            if tool_name in TOOLS:
                result = TOOLS[tool_name](argument)
                print(f"📊 Result: {result}")
                conversation += f"{response}\n\nOBSERVATION: {json.dumps(result)}\n\n"
            else:
                conversation += f"{response}\n\nOBSERVATION: Error - Unknown tool '{tool_name}'\n\n"
        
        else:
            print(f"⚠️ Parse error: {parsed[1]}")
            conversation += f"{response}\n\nOBSERVATION: Please use the correct format.\n\n"
    
    return "Max steps reached without conclusion."

print("✅ Agent loop defined!")

## Step 3: Test the Agent

In [ ]:
# Test 1: Simple IP lookup
print("=" * 60)
print("TEST 1: Is 45.33.32.156 malicious?")
print("=" * 60)

result = run_agent("Is the IP address 45.33.32.156 malicious?")

In [ ]:
# Test 2: Multi-step investigation
print("=" * 60)
print("TEST 2: Multi-indicator investigation")
print("=" * 60)

result = run_agent(
    "I found a suspicious connection to evil.com from IP 192.168.1.100. "
    "Can you investigate both indicators?"
)

## Exercises

### Exercise 1: Add a new tool
Add a `get_whois(domain)` tool that returns mock WHOIS data.

### Exercise 2: Improve error handling
What happens if the LLM doesn't follow the format? Add better error recovery.

### Exercise 3: Add memory
Modify the agent to remember previous investigations in the same session.

## Next Steps

- **Lab 16**: Build a more sophisticated threat intelligence agent with LangChain
- **Lab 18**: Add RAG to give your agent access to documentation
- **Lab 10**: Build a full IR Copilot with state management